In [1]:
# Loading libraries used throughout the notebook
import pandas as pd
import numpy as np

# Reading in the datasets from csv using pandas
dfMockTest = pd.read_csv('TestResults/Formative_Mock_Test.csv', index_col = False)
dfTest_1 = pd.read_csv('TestResults/Formative_Test_1.csv', index_col = False)
dfTest_2 = pd.read_csv('TestResults/Formative_Test_2.csv', index_col = False)
dfTest_3 = pd.read_csv('TestResults/Formative_Test_3.csv', index_col = False)
dfTest_4 = pd.read_csv('TestResults/Formative_Test_4.csv', index_col = False)
dfSumTest = pd.read_csv('TestResults/SumTest.csv', index_col = False)


In [2]:
# Uniform changes made throughout the datasets function
def standard_changes(df):
    
    """Function that adjusts consitent column headings
    for all tests """

    # Remove leading and tailing whitespaces in column headings
    df.columns.str.strip()
    
    # Remove unwanted columns from dataframe
    df.drop(['State', 'Time taken'], axis = 1, inplace = True)
    
    # Rename standard column headings in the datframe
    df.rename(columns = {'research id':'research_id', 'Started on':'started_on', 'Completed':'completed'}, inplace = True)
    
    return df
    

In [3]:
# Clean the question and grade columns (varying elemetns by datset) 
def clean_questions(df):
    
    """Function cleans the question and grade columns
    of the given dataframes. This is achieved by:
    - Splitting and storing total marks available from column heading
    - Removing whitespaces to tidy column headings
    - Rename all the columns to the new clean headings """

    # Splitting and storing total marks in relation to each question
    col_split_index = df.columns.str.split("/")
    col_head = col_split_index[0] + col_split_index[1] + col_split_index[2]
    col_head.append(col_split_index[3][0].lower())
    q_marks = []
    q_heading = []
    
    # Loop thorugh split column headings to fill 'question number' and 'marks available' vectors respectively
    for i in range(4, len(col_split_index)):
        q_heading.append(col_split_index[i][0])
        q_marks.append(col_split_index[i][1])
        i += 1
        
    # Remove whitespaces within question column heading (i.e "Q 1" to become "Q1") using loops
    q_split = []
    for j in q_heading:
        q_split.append(j.split(" "))
    
    # Define vector which contains new clean column headings ('col_head')
    q_split_index = []
    for k in range(0,len(q_split)):
        q_split_index.append(q_split[k][0] + q_split[k][1])
        col_head.append(q_split_index[k])
        k += 1       

    # Rename the dataframe column headings using the vector 'col_head' defined above
    for c in range(0,len(df.columns)):
        df.rename(columns = {df.columns[c]:col_head[c]}, inplace = True)
        c += 1
    
    return df

In [4]:
# Remove missing and null values from data frame
def remove_null(df):
    
    """Function replaces null and '-' values with 0 
    allowing for clean computation """
    
    # Convert 'na' and blank ("-") values to 0 in preparation for value normalisation
    df.fillna(0, inplace = True)
    df.replace(to_replace = ["-"], value = 0, inplace = True)
    
    return df

In [5]:
# Function to adjsut column classifications
def adjust_col_types(df):
    
    """Function to ensure correct classifications by:
    - Converting numbers from strings to 'int64' format
    - Converting dates to 'datetime64[ns]' format
    - Converting marks to 'float64' format """
    
    # Research ID to integer
    df.loc[:,'research_id'] = df.loc[:,'research_id'].apply(pd.to_numeric).astype('int64')
    
    # Date cols to datetime format
    df.loc[:,'started_on':'completed'] = df.loc[:,'started_on':'completed'].apply(pd.to_datetime)
    
    # Marks to floats format
    df.loc[:,'grade':] = df.loc[:,'grade':].apply(pd.to_numeric).astype('float64')
    
    return df

In [6]:
# Remove lowest grade if duplicates exist function
def remove_lowest_grade(df):
    
    """Function removing any duplicate entries and 
    taking the highest grade value if the duplicate exists """

    df = df.sort_values(['grade'], ascending = False).drop_duplicates(['research_id'], keep = 'first')
    
    return df


In [7]:
# 1.2 Clean DataFrame Function
def clean_data(df):
    
    """Function to clean a given dataframe by:
    - Making standard column changes in snake case format
    - Clean variable column headings in snake case format
    - Adjust na and blank values with 0 marks
    - Ensure columns are in the correct datatype
    - Call function to remove duplicate attmepts taking highest grade
    - Order and return the final clean dataframe """

    # Call function for djustment to standard expected column headings
    df = standard_changes(df)

    # Call function for adjustment of variable columns for each dataset
    df = clean_questions(df)
    
    # Call function to remove null values from the dataframe 
    df = remove_null(df)
    
    # Call classification adjustment function
    df = adjust_col_types(df)
    
    # Call function to remove worst result if duplicates exist
    df = remove_lowest_grade(df)
    
    # Order by student reference and return clean dataframe as output
    df = df.sort_values(['research_id'], ascending = True)
    return df



In [8]:
# 1.3 Normalise marks function
def normalise_data(dfClean, dfOriginal):
    
    """Function to normalise grades in the given data frame. 
    This function works by:
    - Taking the clean and the original dataframe then;
    - Cleaning the original to match standard changes
    - Storing a list of total available marks per question
    - Coverting the available marks list to floats
    - Iterating through the question columns to normalise the values in each 
    - Finally order and return """
    
    # Call standard changes function on the original dataframe to match clean data frame
    standard_changes(dfOriginal)
    
    # Splitting and saving marks available for each question from original dataframe
    grade_vector = []
    col_split_index = dfOriginal.columns.str.split("/")
    
    for i in range(dfClean.columns.get_loc('completed') + 1, len(col_split_index)):
        grade_vector.append(col_split_index[i][1])
        i += 1
    
    # Convert total available marks list from string to float (dividing by 100 to obtain raw mark value)
    grade_vector[:] = pd.to_numeric(grade_vector[:], errors='coerce').astype('float64')/100
    
    # Iterate through columns performing value standardisation (making marks a percentage)
    for m in range(dfClean.columns.get_loc('grade'), len(dfClean.columns)):
        dfClean.iloc[:,m] = ((dfClean.iloc[:,m]/grade_vector[m - dfClean.columns.get_loc('grade')]))*100
        m += 1
    
    # Order by student reference and return clean formatted dataframe as output
    dfClean = dfClean.sort_values(['research_id'], ascending = True)
    return dfClean


In [9]:
# Define new names and call the clean data function for specified data frames

# Creating a copy of the dfs
dfCleanMockTest = dfMockTest.copy()
dfCleanTest_1 = dfTest_1.copy()
dfCleanTest_2 = dfTest_2.copy()
dfCleanTest_3 = dfTest_3.copy()
dfCleanTest_4 = dfTest_4.copy()
dfCleanSumTest = dfSumTest.copy()

dfCleanMockTest = clean_data(dfCleanMockTest)
dfCleanTest_1 = clean_data(dfCleanTest_1)
dfCleanTest_2 = clean_data(dfCleanTest_2)
dfCleanTest_3 = clean_data(dfCleanTest_3)
dfCleanTest_4 = clean_data(dfCleanTest_4)
dfCleanSumTest = clean_data(dfCleanSumTest)


In [10]:
# Define new names and call the normalising function for specified data frames

# Creating a copy of the dfs
dfFormattedCleanMockTest = dfCleanMockTest.copy()
dfFormattedCleanTest_1 = dfCleanTest_1.copy()
dfFormattedCleanTest_2 = dfCleanTest_2.copy()
dfFormattedCleanTest_3 = dfCleanTest_3.copy()
dfFormattedCleanTest_4 = dfCleanTest_4.copy()
dfFormattedCleanSumTest = dfCleanSumTest.copy()

dfFormattedCleanMockTest = normalise_data(dfFormattedCleanMockTest, dfMockTest)
dfFormattedCleanTest_1 = normalise_data(dfFormattedCleanTest_1, dfTest_1)
dfFormattedCleanTest_2 = normalise_data(dfFormattedCleanTest_2, dfTest_2)
dfFormattedCleanTest_3 = normalise_data(dfFormattedCleanTest_3, dfTest_3)
dfFormattedCleanTest_4 = normalise_data(dfFormattedCleanTest_4, dfTest_4)
dfFormattedCleanSumTest = normalise_data(dfFormattedCleanSumTest, dfSumTest)


In [11]:
# Example of Formatted Clean Dataframe for SumTest
print(dfFormattedCleanSumTest)

     research_id          started_on           completed   grade     Q1  \
32             1 2018-11-14 09:50:00 2018-11-14 10:51:00   66.00  100.0   
45             2 2018-11-14 09:50:00 2018-11-14 10:31:00  100.00  100.0   
7              3 2018-11-14 09:50:00 2018-11-14 10:51:00   63.75  100.0   
120            4 2018-11-14 11:09:00 2018-11-14 12:10:00   51.00    0.0   
143            5 2018-11-14 11:10:00 2018-11-14 12:11:00   46.63  100.0   
..           ...                 ...                 ...     ...    ...   
68           152 2018-11-14 09:50:00 2018-11-14 10:51:00   72.00  100.0   
53           153 2018-11-14 09:50:00 2018-11-14 10:51:00   45.00  100.0   
78           154 2018-11-14 11:09:00 2018-11-14 12:10:00   94.00  100.0   
14           155 2018-11-14 09:50:00 2018-11-14 10:51:00   95.00  100.0   
3            156 2018-11-14 09:50:00 2018-11-14 10:51:00   64.00  100.0   

        Q2     Q3     Q4     Q5     Q6     Q7          Q8          Q9    Q10  \
32   100.0  100.0  

In [12]:
import sqlite3

# Define SQL connection
results = sqlite3.connect('Resultdatabase.db')

# Send data frames to SQL tables
dfFormattedCleanMockTest.to_sql("dfFormattedCleanMockTest", results, if_exists='replace', index = False)
dfFormattedCleanTest_1.to_sql("dfFormattedCleanTest_1", results, if_exists='replace', index = False)
dfFormattedCleanTest_2.to_sql("dfFormattedCleanTest_2", results, if_exists='replace', index = False)
dfFormattedCleanTest_3.to_sql("dfFormattedCleanTest_3", results, if_exists='replace', index = False)
dfFormattedCleanTest_4.to_sql("dfFormattedCleanTest_4", results, if_exists='replace', index = False)
dfFormattedCleanSumTest.to_sql("dfFormattedCleanSumTest", results, if_exists='replace', index = False)

# Autosave (commit) and close the database file 
results.close()


In [13]:
# Create a function to randomise and overwrite marks
def modify_data(dbFile, index_no, question_no):
    
    """Function creates a random vector to represent the 
    number of indexes to modify, which questions and how many 
    to modify. It also randomly assigns new grades then finally
    updates the database file storing the results."""

    # Create random number vector of specified ('index_no') number of unique values between 0 and 150
    flag_0 = True
    while flag_0 == True:   
        indicies = np.random.randint(low = 0, high = 150, size = index_no)
        indicies = list(dict.fromkeys(indicies))
        indicies.sort()
        if len(indicies) != index_no:
            flag_0 = True
        else:
            flag_0 = False
            break 

    # Create unique random column index for specified number ('question_no') of questions in order to edit the results
    flag_1 = True
    while flag_1 == True:
        questions = np.random.randint(low = 4, high = 17, size = question_no)
        questions = list(dict.fromkeys(questions))
        questions.sort()
        if len(questions) != question_no:
            flag_1 = True
        else:
            flag_1 = False
            break 


    # Create a new dataframe storing random grades as percentages
    mod_grade_col = []
    for c in range(0, question_no):
        mod_grade_col.append(str('mod' + str(c)))
        c += 1

    mod_grades = pd.DataFrame(index = np.arange(index_no), columns = mod_grade_col)    
    for j in range(0,len(questions)):
        mod_grades.iloc[:,j] = np.round_(np.random.uniform(low = 0, high = 1, size = index_no), decimals = 3)*100


    # Update the main dataframe with the modified values
    results = sqlite3.connect(dbFile)

    dfFormattedCleanSumTest = pd.read_sql('SELECT * FROM dfFormattedCleanSumTest', results)
    
    for i in range(0,len(mod_grades)):
        for j in range(0,len(mod_grades.columns)):
            dfFormattedCleanSumTest.iloc[indicies[i], questions[j]] = mod_grades.iloc[i,j]
            
        new_total = dfFormattedCleanSumTest.iloc[indicies[i], dfFormattedCleanSumTest.columns.get_loc('grade')+1:].sum()
        total_qs = len(dfFormattedCleanSumTest.columns) - (dfFormattedCleanSumTest.columns.get_loc('grade')+1)

        dfFormattedCleanSumTest.iloc[indicies[i], dfFormattedCleanSumTest.columns.get_loc('grade')] = new_total / total_qs
    
    
    #Save the modified data over original dataframe
    dfFormattedCleanSumTest.to_sql("dfFormattedCleanSumTest", results, if_exists='replace', index = False)

    # Autosave (commit) and close the database file 
    results.close()


In [14]:
# Call modify data function for 20 rows changing marks for a random 3 questions
modify_data('Resultdatabase.db', 20, 3)

In [1]:
# Check database saves values
import sqlite3
import pandas as pd 

results = sqlite3.connect('Resultdatabase.db')

modified = pd.read_sql('SELECT * FROM dfFormattedCleanTest_2 WHERE research_id == 122', results)
print(modified.iloc[:, :])
print(modified.empty)

results.close()

   research_id           started_on            completed      grade    Q1  \
0          122  2018-10-17 10:15:00  2018-10-18 13:05:00  91.428571  60.0   

      Q2     Q3     Q4    Q5     Q6  
0  100.0  100.0  100.0  80.0  100.0  
False
